In [ ]:
import sys, os, pickle
from joblib import Parallel, delayed
import pandas as pd
from jumpmodels.utils import filter_date_range
from jumpmodels.preprocess import StandardScalerPD, DataClipperStd
from jumpmodels.sparse_jump import SparseJumpModel
from sklearn.preprocessing import StandardScaler
from pypfopt.black_litterman import BlackLittermanModel, market_implied_prior_returns
from pypfopt.efficient_frontier import EfficientFrontier
import numpy as np

from feature_set_v2 import MergedDataLoader   
#sys.path.append('/Users/victor/Documents/thesis_vri_vp/vic_new')         # for mac
sys.path.append('C:\\Users\\victo\\git_new\\thesis_vri_vp\\vic_new')      # for windows

In [ ]:
hyperparameter_history = {
    'iwf': [
        {'date': pd.Timestamp('2017-01-31 00:00:00'), 'old_lambda': None, 'new_lambda': 194.44444444444446, 'old_kappa': None, 'new_kappa': 2.802775637731995, 'sharpe': -0.00040810662925682057},
        {'date': pd.Timestamp('2017-07-31 00:00:00'), 'old_lambda': 194.44444444444446, 'new_lambda': 194.44444444444446, 'old_kappa': 2.802775637731995, 'new_kappa': 2.0, 'sharpe': 0.17071973179690386},
        {'date': pd.Timestamp('2018-01-31 00:00:00'), 'old_lambda': 194.44444444444446, 'new_lambda': 358.8888888888889, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 0.3578102263098243},
        {'date': pd.Timestamp('2018-07-31 00:00:00'), 'old_lambda': 358.8888888888889, 'new_lambda': 358.8888888888889, 'old_kappa': 2.0, 'new_kappa': 2.802775637731995, 'sharpe': 0.4841917963037153},
        {'date': pd.Timestamp('2019-01-31 00:00:00'), 'old_lambda': 358.8888888888889, 'new_lambda': 358.8888888888889, 'old_kappa': 2.802775637731995, 'new_kappa': 2.0, 'sharpe': 0.0021833092539811937},
        {'date': pd.Timestamp('2019-07-31 00:00:00'), 'old_lambda': 358.8888888888889, 'new_lambda': 400.0, 'old_kappa': 2.0, 'new_kappa': 2.802775637731995, 'sharpe': -0.08661718291998324},
        {'date': pd.Timestamp('2020-01-31 00:00:00'), 'old_lambda': 400.0, 'new_lambda': 400.0, 'old_kappa': 2.802775637731995, 'new_kappa': 2.802775637731995, 'sharpe': 0.04537836286365537},
        {'date': pd.Timestamp('2020-07-31 00:00:00'), 'old_lambda': 400.0, 'new_lambda': 400.0, 'old_kappa': 2.802775637731995, 'new_kappa': 2.802775637731995, 'sharpe': 0.3796416073101208},
        {'date': pd.Timestamp('2021-01-29 00:00:00'), 'old_lambda': 400.0, 'new_lambda': 400.0, 'old_kappa': 2.802775637731995, 'new_kappa': 2.0, 'sharpe': 0.3143905966885326},
        {'date': pd.Timestamp('2021-07-30 00:00:00'), 'old_lambda': 400.0, 'new_lambda': 71.11111111111111, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 0.6565757223197191},
        {'date': pd.Timestamp('2022-01-31 00:00:00'), 'old_lambda': 71.11111111111111, 'new_lambda': 30.0, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 0.5661972679926426},
        {'date': pd.Timestamp('2022-07-29 00:00:00'), 'old_lambda': 30.0, 'new_lambda': 194.44444444444446, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 0.5025574640511644},
        {'date': pd.Timestamp('2023-01-31 00:00:00'), 'old_lambda': 194.44444444444446, 'new_lambda': 194.44444444444446, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 0.4412874588946835},
        {'date': pd.Timestamp('2023-07-31 00:00:00'), 'old_lambda': 194.44444444444446, 'new_lambda': 112.22222222222223, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 0.6244571972898924},
        {'date': pd.Timestamp('2024-01-31 00:00:00'), 'old_lambda': 112.22222222222223, 'new_lambda': 112.22222222222223, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 0.5496996539335203},
        {'date': pd.Timestamp('2024-07-31 00:00:00'), 'old_lambda': 112.22222222222223, 'new_lambda': 112.22222222222223, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 0.5045344785461053},
        {'date': pd.Timestamp('2025-01-31 00:00:00'), 'old_lambda': 112.22222222222223, 'new_lambda': 153.33333333333334, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 0.6748125011222248},
        {'date': pd.Timestamp('2025-04-03 00:00:00'), 'old_lambda': 153.33333333333334, 'new_lambda': 112.22222222222223, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 0.5220707952568912}
    ],
    'mtum': [
        {'date': pd.Timestamp('2017-01-31 00:00:00'), 'old_lambda': None, 'new_lambda': 153.33333333333334, 'old_kappa': None, 'new_kappa': 3.605551275463989, 'sharpe': 0.9737531059727994},
        {'date': pd.Timestamp('2017-07-31 00:00:00'), 'old_lambda': 153.33333333333334, 'new_lambda': 71.11111111111111, 'old_kappa': 3.605551275463989, 'new_kappa': 2.0, 'sharpe': 1.2385288514186117},
        {'date': pd.Timestamp('2018-01-31 00:00:00'), 'old_lambda': 71.11111111111111, 'new_lambda': 71.11111111111111, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 1.5384280936967225},
        {'date': pd.Timestamp('2018-07-31 00:00:00'), 'old_lambda': 71.11111111111111, 'new_lambda': 71.11111111111111, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 1.2449318483644662},
        {'date': pd.Timestamp('2019-01-31 00:00:00'), 'old_lambda': 71.11111111111111, 'new_lambda': 112.22222222222223, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 1.2849216385777595},
        {'date': pd.Timestamp('2019-07-31 00:00:00'), 'old_lambda': 112.22222222222223, 'new_lambda': 112.22222222222223, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 1.3550258164181153},
        {'date': pd.Timestamp('2020-01-31 00:00:00'), 'old_lambda': 112.22222222222223, 'new_lambda': 112.22222222222223, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 1.2946985716778718},
        {'date': pd.Timestamp('2020-07-31 00:00:00'), 'old_lambda': 112.22222222222223, 'new_lambda': 112.22222222222223, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 1.2443701401904304},
        {'date': pd.Timestamp('2021-01-29 00:00:00'), 'old_lambda': 112.22222222222223, 'new_lambda': 30.0, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 1.0609358262039597},
        {'date': pd.Timestamp('2021-07-30 00:00:00'), 'old_lambda': 30.0, 'new_lambda': 317.7777777777778, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 0.5336681869528377},
        {'date': pd.Timestamp('2022-01-31 00:00:00'), 'old_lambda': 317.7777777777778, 'new_lambda': 71.11111111111111, 'old_kappa': 2.0, 'new_kappa': 2.802775637731995, 'sharpe': 0.42085014436258855},
        {'date': pd.Timestamp('2022-07-29 00:00:00'), 'old_lambda': 71.11111111111111, 'new_lambda': 71.11111111111111, 'old_kappa': 2.802775637731995, 'new_kappa': 2.802775637731995, 'sharpe': 0.4776764452885165},
        {'date': pd.Timestamp('2023-01-31 00:00:00'), 'old_lambda': 71.11111111111111, 'new_lambda': 30.0, 'old_kappa': 2.802775637731995, 'new_kappa': 3.605551275463989, 'sharpe': 0.29425697780975163},
        {'date': pd.Timestamp('2023-07-31 00:00:00'), 'old_lambda': 30.0, 'new_lambda': 30.0, 'old_kappa': 3.605551275463989, 'new_kappa': 3.605551275463989, 'sharpe': 0.12389367197312393},
        {'date': pd.Timestamp('2024-01-31 00:00:00'), 'old_lambda': 30.0, 'new_lambda': 235.55555555555557, 'old_kappa': 3.605551275463989, 'new_kappa': 2.802775637731995, 'sharpe': 0.30857404399436067},
        {'date': pd.Timestamp('2024-07-31 00:00:00'), 'old_lambda': 235.55555555555557, 'new_lambda': 358.8888888888889, 'old_kappa': 2.802775637731995, 'new_kappa': 2.0, 'sharpe': 0.39916193058999083},
        {'date': pd.Timestamp('2025-01-31 00:00:00'), 'old_lambda': 358.8888888888889, 'new_lambda': 276.6666666666667, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 0.29269886811293266},
        {'date': pd.Timestamp('2025-04-03 00:00:00'), 'old_lambda': 276.6666666666667, 'new_lambda': 358.8888888888889, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 0.48260717116695645}
    ],
    'qual': [
        {'date': pd.Timestamp('2017-01-31 00:00:00'), 'old_lambda': None, 'new_lambda': 112.22222222222223, 'old_kappa': None, 'new_kappa': 2.0, 'sharpe': -0.008942944608080107},
        {'date': pd.Timestamp('2017-07-31 00:00:00'), 'old_lambda': 112.22222222222223, 'new_lambda': 194.44444444444446, 'old_kappa': 2.0, 'new_kappa': 2.802775637731995, 'sharpe': 0.3729840213884896},
        {'date': pd.Timestamp('2018-01-31 00:00:00'), 'old_lambda': 194.44444444444446, 'new_lambda': 276.6666666666667, 'old_kappa': 2.802775637731995, 'new_kappa': 2.0, 'sharpe': 0.4569557693250369},
        {'date': pd.Timestamp('2018-07-31 00:00:00'), 'old_lambda': 276.6666666666667, 'new_lambda': 276.6666666666667, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 0.46645808224234},
        {'date': pd.Timestamp('2019-01-31 00:00:00'), 'old_lambda': 276.6666666666667, 'new_lambda': 317.7777777777778, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 0.44970951739981563},
        {'date': pd.Timestamp('2019-07-31 00:00:00'), 'old_lambda': 317.7777777777778, 'new_lambda': 358.8888888888889, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 0.4565696219766751},
        {'date': pd.Timestamp('2020-01-31 00:00:00'), 'old_lambda': 358.8888888888889, 'new_lambda': 276.6666666666667, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 0.515467869569287},
        {'date': pd.Timestamp('2020-07-31 00:00:00'), 'old_lambda': 276.6666666666667, 'new_lambda': 30.0, 'old_kappa': 2.0, 'new_kappa': 2.802775637731995, 'sharpe': 0.34082022634710635},
        {'date': pd.Timestamp('2021-01-29 00:00:00'), 'old_lambda': 30.0, 'new_lambda': 30.0, 'old_kappa': 2.802775637731995, 'new_kappa': 3.605551275463989, 'sharpe': 0.43043837227656606},
        {'date': pd.Timestamp('2021-07-30 00:00:00'), 'old_lambda': 30.0, 'new_lambda': 235.55555555555557, 'old_kappa': 3.605551275463989, 'new_kappa': 2.0, 'sharpe': 0.48545612296409024},
        {'date': pd.Timestamp('2022-01-31 00:00:00'), 'old_lambda': 235.55555555555557, 'new_lambda': 235.55555555555557, 'old_kappa': 2.0, 'new_kappa': 2.802775637731995, 'sharpe': 0.24680166988096894},
        {'date': pd.Timestamp('2022-07-29 00:00:00'), 'old_lambda': 235.55555555555557, 'new_lambda': 194.44444444444446, 'old_kappa': 2.802775637731995, 'new_kappa': 2.0, 'sharpe': 0.3768471604383053},
        {'date': pd.Timestamp('2023-01-31 00:00:00'), 'old_lambda': 194.44444444444446, 'new_lambda': 235.55555555555557, 'old_kappa': 2.0, 'new_kappa': 3.605551275463989, 'sharpe': 0.52515080472428},
        {'date': pd.Timestamp('2023-07-31 00:00:00'), 'old_lambda': 235.55555555555557, 'new_lambda': 276.6666666666667, 'old_kappa': 3.605551275463989, 'new_kappa': 3.605551275463989, 'sharpe': 0.5032881257329227},
        {'date': pd.Timestamp('2024-01-31 00:00:00'), 'old_lambda': 276.6666666666667, 'new_lambda': 276.6666666666667, 'old_kappa': 3.605551275463989, 'new_kappa': 2.0, 'sharpe': 0.5601170142538924},
        {'date': pd.Timestamp('2024-07-31 00:00:00'), 'old_lambda': 276.6666666666667, 'new_lambda': 30.0, 'old_kappa': 2.0, 'new_kappa': 3.605551275463989, 'sharpe': 0.6556926212774876},
        {'date': pd.Timestamp('2025-01-31 00:00:00'), 'old_lambda': 30.0, 'new_lambda': 30.0, 'old_kappa': 3.605551275463989, 'new_kappa': 2.802775637731995, 'sharpe': 0.6397538574423197},
        {'date': pd.Timestamp('2025-04-03 00:00:00'), 'old_lambda': 30.0, 'new_lambda': 30.0, 'old_kappa': 2.802775637731995, 'new_kappa': 3.605551275463989, 'sharpe': 0.595839463815995}
    ],
    'size': [
        {'date': pd.Timestamp('2017-01-31 00:00:00'), 'old_lambda': None, 'new_lambda': 30.0, 'old_kappa': None, 'new_kappa': 3.605551275463989, 'sharpe': 0.18325758924372446},
        {'date': pd.Timestamp('2017-07-31 00:00:00'), 'old_lambda': 30.0, 'new_lambda': 30.0, 'old_kappa': 3.605551275463989, 'new_kappa': 3.605551275463989, 'sharpe': 0.35998093208599363},
        {'date': pd.Timestamp('2018-01-31 00:00:00'), 'old_lambda': 30.0, 'new_lambda': 30.0, 'old_kappa': 3.605551275463989, 'new_kappa': 2.802775637731995, 'sharpe': 0.20754182203863084},
        {'date': pd.Timestamp('2018-07-31 00:00:00'), 'old_lambda': 30.0, 'new_lambda': 30.0, 'old_kappa': 2.802775637731995, 'new_kappa': 3.605551275463989, 'sharpe': 0.169026897105994},
        {'date': pd.Timestamp('2019-01-31 00:00:00'), 'old_lambda': 30.0, 'new_lambda': 30.0, 'old_kappa': 3.605551275463989, 'new_kappa': 3.605551275463989, 'sharpe': 0.170613601033382},
        {'date': pd.Timestamp('2019-07-31 00:00:00'), 'old_lambda': 30.0, 'new_lambda': 30.0, 'old_kappa': 3.605551275463989, 'new_kappa': 3.605551275463989, 'sharpe': 0.08232751636587375},
        {'date': pd.Timestamp('2020-01-31 00:00:00'), 'old_lambda': 30.0, 'new_lambda': 194.44444444444446, 'old_kappa': 3.605551275463989, 'new_kappa': 3.605551275463989, 'sharpe': 0.07975853398451298},
        {'date': pd.Timestamp('2020-07-31 00:00:00'), 'old_lambda': 194.44444444444446, 'new_lambda': 194.44444444444446, 'old_kappa': 3.605551275463989, 'new_kappa': 3.605551275463989, 'sharpe': 0.20993440803949895},
        {'date': pd.Timestamp('2021-01-29 00:00:00'), 'old_lambda': 194.44444444444446, 'new_lambda': 235.55555555555557, 'old_kappa': 3.605551275463989, 'new_kappa': 3.605551275463989, 'sharpe': 0.16038247250831156},
        {'date': pd.Timestamp('2021-07-30 00:00:00'), 'old_lambda': 235.55555555555557, 'new_lambda': 317.7777777777778, 'old_kappa': 3.605551275463989, 'new_kappa': 2.0, 'sharpe': -0.018202918861167948},
        {'date': pd.Timestamp('2022-01-31 00:00:00'), 'old_lambda': 317.7777777777778, 'new_lambda': 400.0, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 0.036480646039279585},
        {'date': pd.Timestamp('2022-07-29 00:00:00'), 'old_lambda': 400.0, 'new_lambda': 358.8888888888889, 'old_kappa': 2.0, 'new_kappa': 2.802775637731995, 'sharpe': 0.36563761781827314},
        {'date': pd.Timestamp('2023-01-31 00:00:00'), 'old_lambda': 358.8888888888889, 'new_lambda': 317.7777777777778, 'old_kappa': 2.802775637731995, 'new_kappa': 2.0, 'sharpe': 0.32185546293215544},
        {'date': pd.Timestamp('2023-07-31 00:00:00'), 'old_lambda': 317.7777777777778, 'new_lambda': 317.7777777777778, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 0.2798812792489098},
        {'date': pd.Timestamp('2024-01-31 00:00:00'), 'old_lambda': 317.7777777777778, 'new_lambda': 358.8888888888889, 'old_kappa': 2.0, 'new_kappa': 2.802775637731995, 'sharpe': 0.452087066336667},
        {'date': pd.Timestamp('2024-07-31 00:00:00'), 'old_lambda': 358.8888888888889, 'new_lambda': 358.8888888888889, 'old_kappa': 2.802775637731995, 'new_kappa': 2.802775637731995, 'sharpe': 0.5006537077202152},
        {'date': pd.Timestamp('2025-01-31 00:00:00'), 'old_lambda': 358.8888888888889, 'new_lambda': 317.7777777777778, 'old_kappa': 2.802775637731995, 'new_kappa': 2.802775637731995, 'sharpe': 0.5716957249478238},
        {'date': pd.Timestamp('2025-04-03 00:00:00'), 'old_lambda': 317.7777777777778, 'new_lambda': 317.7777777777778, 'old_kappa': 2.802775637731995, 'new_kappa': 2.802775637731995, 'sharpe': 0.4431237089045836}
    ],
    'usmv': [
        {'date': pd.Timestamp('2017-01-31 00:00:00'), 'old_lambda': None, 'new_lambda': 30.0, 'old_kappa': None, 'new_kappa': 2.0, 'sharpe': -0.018377230113554608},
        {'date': pd.Timestamp('2017-07-31 00:00:00'), 'old_lambda': 30.0, 'new_lambda': 400.0, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 0.22709101622189878},
        {'date': pd.Timestamp('2018-01-31 00:00:00'), 'old_lambda': 400.0, 'new_lambda': 194.44444444444446, 'old_kappa': 2.0, 'new_kappa': 2.802775637731995, 'sharpe': -0.11574479947038052},
        {'date': pd.Timestamp('2018-07-31 00:00:00'), 'old_lambda': 194.44444444444446, 'new_lambda': 194.44444444444446, 'old_kappa': 2.802775637731995, 'new_kappa': 2.802775637731995, 'sharpe': -0.13169461167089427},
        {'date': pd.Timestamp('2019-01-31 00:00:00'), 'old_lambda': 194.44444444444446, 'new_lambda': 112.22222222222223, 'old_kappa': 2.802775637731995, 'new_kappa': 2.802775637731995, 'sharpe': -0.04023257907985476},
        {'date': pd.Timestamp('2019-07-31 00:00:00'), 'old_lambda': 112.22222222222223, 'new_lambda': 71.11111111111111, 'old_kappa': 2.802775637731995, 'new_kappa': 2.802775637731995, 'sharpe': -0.09485108586318038},
        {'date': pd.Timestamp('2020-01-31 00:00:00'), 'old_lambda': 71.11111111111111, 'new_lambda': 30.0, 'old_kappa': 2.802775637731995, 'new_kappa': 2.0, 'sharpe': -0.20505014291457693},
        {'date': pd.Timestamp('2020-07-31 00:00:00'), 'old_lambda': 30.0, 'new_lambda': 30.0, 'old_kappa': 2.0, 'new_kappa': 2.802775637731995, 'sharpe': -0.21124341082069295},
        {'date': pd.Timestamp('2021-01-29 00:00:00'), 'old_lambda': 30.0, 'new_lambda': 30.0, 'old_kappa': 2.802775637731995, 'new_kappa': 2.802775637731995, 'sharpe': 0.27357211138032833},
        {'date': pd.Timestamp('2021-07-30 00:00:00'), 'old_lambda': 30.0, 'new_lambda': 30.0, 'old_kappa': 2.802775637731995, 'new_kappa': 2.802775637731995, 'sharpe': 0.12318043827830945},
        {'date': pd.Timestamp('2022-01-31 00:00:00'), 'old_lambda': 30.0, 'new_lambda': 30.0, 'old_kappa': 2.802775637731995, 'new_kappa': 3.605551275463989, 'sharpe': -0.027488565563862734},
        {'date': pd.Timestamp('2022-07-29 00:00:00'), 'old_lambda': 30.0, 'new_lambda': 30.0, 'old_kappa': 3.605551275463989, 'new_kappa': 2.802775637731995, 'sharpe': 0.00615644148977025},
        {'date': pd.Timestamp('2023-01-31 00:00:00'), 'old_lambda': 30.0, 'new_lambda': 276.6666666666667, 'old_kappa': 2.802775637731995, 'new_kappa': 2.802775637731995, 'sharpe': 0.1102054318041255},
        {'date': pd.Timestamp('2023-07-31 00:00:00'), 'old_lambda': 276.6666666666667, 'new_lambda': 153.33333333333334, 'old_kappa': 2.802775637731995, 'new_kappa': 2.802775637731995, 'sharpe': 0.18972149133249572},
        {'date': pd.Timestamp('2024-01-31 00:00:00'), 'old_lambda': 153.33333333333334, 'new_lambda': 30.0, 'old_kappa': 2.802775637731995, 'new_kappa': 2.802775637731995, 'sharpe': 0.09270063511915752},
        {'date': pd.Timestamp('2024-07-31 00:00:00'), 'old_lambda': 30.0, 'new_lambda': 30.0, 'old_kappa': 2.802775637731995, 'new_kappa': 2.802775637731995, 'sharpe': 0.23279385139850495},
        {'date': pd.Timestamp('2025-01-31 00:00:00'), 'old_lambda': 30.0, 'new_lambda': 153.33333333333334, 'old_kappa': 2.802775637731995, 'new_kappa': 2.802775637731995, 'sharpe': 0.30721578219158546},
        {'date': pd.Timestamp('2025-04-03 00:00:00'), 'old_lambda': 153.33333333333334, 'new_lambda': 153.33333333333334, 'old_kappa': 2.802775637731995, 'new_kappa': 2.0, 'sharpe': 0.25005959115900506}
    ],
    'vlue': [
        {'date': pd.Timestamp('2017-01-31 00:00:00'), 'old_lambda': None, 'new_lambda': 112.22222222222223, 'old_kappa': None, 'new_kappa': 2.0, 'sharpe': 0.9808698623368385},
        {'date': pd.Timestamp('2017-07-31 00:00:00'), 'old_lambda': 112.22222222222223, 'new_lambda': 317.7777777777778, 'old_kappa': 2.0, 'new_kappa': 3.605551275463989, 'sharpe': 0.8394277858533714},
        {'date': pd.Timestamp('2018-01-31 00:00:00'), 'old_lambda': 317.7777777777778, 'new_lambda': 276.6666666666667, 'old_kappa': 3.605551275463989, 'new_kappa': 2.0, 'sharpe': 1.006140466878844},
        {'date': pd.Timestamp('2018-07-31 00:00:00'), 'old_lambda': 276.6666666666667, 'new_lambda': 112.22222222222223, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 1.0366275635711821},
        {'date': pd.Timestamp('2019-01-31 00:00:00'), 'old_lambda': 112.22222222222223, 'new_lambda': 112.22222222222223, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 1.0882139211247348},
        {'date': pd.Timestamp('2019-07-31 00:00:00'), 'old_lambda': 112.22222222222223, 'new_lambda': 358.8888888888889, 'old_kappa': 2.0, 'new_kappa': 3.605551275463989, 'sharpe': 0.6059643491080812},
        {'date': pd.Timestamp('2020-01-31 00:00:00'), 'old_lambda': 358.8888888888889, 'new_lambda': 358.8888888888889, 'old_kappa': 3.605551275463989, 'new_kappa': 2.802775637731995, 'sharpe': 0.39346946308603115},
        {'date': pd.Timestamp('2020-07-31 00:00:00'), 'old_lambda': 358.8888888888889, 'new_lambda': 153.33333333333334, 'old_kappa': 2.802775637731995, 'new_kappa': 3.605551275463989, 'sharpe': 0.350841470339303},
        {'date': pd.Timestamp('2021-01-29 00:00:00'), 'old_lambda': 153.33333333333334, 'new_lambda': 153.33333333333334, 'old_kappa': 3.605551275463989, 'new_kappa': 3.605551275463989, 'sharpe': 0.4540450646485634},
        {'date': pd.Timestamp('2021-07-30 00:00:00'), 'old_lambda': 153.33333333333334, 'new_lambda': 235.55555555555557, 'old_kappa': 3.605551275463989, 'new_kappa': 2.802775637731995, 'sharpe': 0.45721111621311916},
        {'date': pd.Timestamp('2022-01-31 00:00:00'), 'old_lambda': 235.55555555555557, 'new_lambda': 112.22222222222223, 'old_kappa': 2.802775637731995, 'new_kappa': 2.802775637731995, 'sharpe': 0.4393294952136704},
        {'date': pd.Timestamp('2022-07-29 00:00:00'), 'old_lambda': 112.22222222222223, 'new_lambda': 153.33333333333334, 'old_kappa': 2.802775637731995, 'new_kappa': 2.802775637731995, 'sharpe': 0.538921576147189},
        {'date': pd.Timestamp('2023-01-31 00:00:00'), 'old_lambda': 153.33333333333334, 'new_lambda': 112.22222222222223, 'old_kappa': 2.802775637731995, 'new_kappa': 3.605551275463989, 'sharpe': 0.3856975910925221},
        {'date': pd.Timestamp('2023-07-31 00:00:00'), 'old_lambda': 112.22222222222223, 'new_lambda': 153.33333333333334, 'old_kappa': 3.605551275463989, 'new_kappa': 2.802775637731995, 'sharpe': 0.3300623151991221},
        {'date': pd.Timestamp('2024-01-31 00:00:00'), 'old_lambda': 153.33333333333334, 'new_lambda': 153.33333333333334, 'old_kappa': 2.802775637731995, 'new_kappa': 3.605551275463989, 'sharpe': 0.31531828237806936},
        {'date': pd.Timestamp('2024-07-31 00:00:00'), 'old_lambda': 153.33333333333334, 'new_lambda': 112.22222222222223, 'old_kappa': 3.605551275463989, 'new_kappa': 2.802775637731995, 'sharpe': 0.22277643742023429},
        {'date': pd.Timestamp('2025-01-31 00:00:00'), 'old_lambda': 112.22222222222223, 'new_lambda': 400.0, 'old_kappa': 2.802775637731995, 'new_kappa': 2.0, 'sharpe': -0.059024223017440335},
        {'date': pd.Timestamp('2025-04-03 00:00:00'), 'old_lambda': 400.0, 'new_lambda': 400.0, 'old_kappa': 2.0, 'new_kappa': 2.0, 'sharpe': 0.05808658892901897}
    ]
}


In [ ]:
def format_saved_hyperparameters(h):
    return h        # identical to old behaviour

saved_hyperparams = format_saved_hyperparameters(hyperparameter_history)

In [ ]:
# ──────────────────────────────────────────────────────────────
# DATA‑LOADING BLOCK  (pulled from old notebook)
# ──────────────────────────────────────────────────────────────

# 0) Global parameters ------------------------------------------------------------------
REFIT_FREQ        = "MS"        # must match new script
MIN_TRAINING_YEARS= 8
MAX_TRAINING_YEARS= 12
INITIAL_TRAIN_START = "2002-05-31"
test_start        = "2017-01-01"

# Paths & tickers -----------------------------------------------------------------------
script_dir = os.getcwd()
base_dir   = os.path.abspath(os.path.join(script_dir, "..", ".."))
data_dir   = os.path.join(base_dir, "data_new")

factor_file = os.path.join(data_dir, "1estimation_index_returns.csv")
market_file = os.path.join(data_dir, "1macro_data.csv")
etf_file    = os.path.join(data_dir, "2trading_etf_returns_aligned.csv")

factors = ["iwf", "mtum", "qual", "size", "usmv", "vlue"]   # used everywhere

# 1) Load full data for every factor + market ------------------------------------------------
factor_data_dict  = {}
factor_returns_ls = []

for fac in factors:
    print(f"Loading data for {fac}")
    data = MergedDataLoader(
        factor_file=factor_file,
        market_file=market_file,
        ver="v2",
        factor_col=fac
    ).load()

    common_idx = (data.X.index
                  .intersection(data.ret_ser.index)
                  .intersection(data.market_ser.index))

    X_full        = data.X.loc[common_idx]
    fac_ret_full  = data.ret_ser.loc[common_idx]
    mkt_ret_full  = data.market_ser.loc[common_idx]
    active_ret    = fac_ret_full - mkt_ret_full

    factor_data_dict[fac] = {
        "X"        : X_full,
        "fac_ret"  : fac_ret_full,
        "mkt_ret"  : mkt_ret_full,
        "active_ret": active_ret,
    }
    factor_returns_ls.append(fac_ret_full)

# save last loop’s mkt_ret_full as market series
all_market_ret = mkt_ret_full

# 2) Assemble master return dataframe (factors + Market + rf) -------------------------------
full_factors_df = pd.concat(factor_returns_ls, axis=1).dropna()
full_df = pd.concat([full_factors_df, all_market_ret], axis=1).dropna()
full_df.columns = factors + ["Market"]

# risk‑free
etf_df   = pd.read_csv(etf_file, index_col=0, parse_dates=True).dropna().sort_index()
rf_ser   = etf_df["rf"]
full_df  = pd.concat([full_df, rf_ser], axis=1).dropna()
full_df.columns = factors + ["Market", "rf"]

# 3) Define test index (everything from 2017‑01‑01 on) --------------------------------------
test_slice = full_df.loc[test_start:]
test_index = test_slice.index.sort_values()
# ──────────────────────────────────────────────────────────────


In [ ]:
# ------------------------------------------------------------
# ①  BUILD & CACHE FACTOR‑VIEWS  (run once, takes minutes)
# ------------------------------------------------------------
VIEWS_FILE = "factor_views.pkl"          # change path if you like
FORCE_REBUILD = True 

def _fit_one_factor(fac, refit_date, test_dates_chunk,
                    factor_data_dict, hyperparams,
                    min_years, max_years, init_start):

    # ---------- helpers ----------
    def get_train_window(current_date, full_data):
        train_end  = current_date
        train_start= max(train_end - pd.DateOffset(years=max_years),
                         pd.to_datetime(init_start))
        if (train_end - train_start) < pd.Timedelta(days=365.25*min_years):
            train_start = train_end - pd.DateOffset(years=min_years)
        idx = full_data.index
        mask = (idx>=train_start)&(idx<=train_end)
        return idx[mask][0], idx[mask][-1]

    # ---------- data ----------
    fac_data = factor_data_dict[fac]
    X   = fac_data["X"]
    ret = fac_data["fac_ret"]
    act = fac_data["active_ret"]

    lam = hyperparams["new_lambda"]
    kp  = hyperparams["new_kappa"]
    train_start, train_end = get_train_window(refit_date, X)

    # ---------- preprocess ----------
    clipper = DataClipperStd(mul=3.0)
    scaler  = StandardScaler()
    X_train = scaler.fit_transform(clipper.fit_transform(
                 filter_date_range(X, train_start, train_end)))
    active_train = filter_date_range(act, train_start, train_end)

    # ---------- fit SJM ----------
    sjm = SparseJumpModel(n_components=2,
                          max_feats=int(kp**2),
                          jump_penalty=lam)
    
    train_idx = filter_date_range(X, train_start, train_end).index
    X_train_df = pd.DataFrame(X_train, index=train_idx, columns=X.columns)
    sjm.fit(X_train_df, ret_ser=active_train, sort_by="cumret")

    ret_train = filter_date_range(ret, train_start, train_end)

    # regime‑level abs returns
    train_states = sjm.predict(X_train_df)
    abs_ret = {}
    for st in range(2):
        st_idx = (train_states==st)
        abs_ret[st] = ret_train.loc[st_idx].mean() * 252

    # ---------- online prediction for test dates ----------
    states = {}
    for day in test_dates_chunk:
        X_hist = X.loc[:day]                          # all history up to 'day'
        temp_clipper = DataClipperStd(mul=3.0)
        X_hist_clip  = temp_clipper.fit_transform(X_hist)

    temp_scaler  = StandardScaler()
    _ = temp_scaler.fit_transform(X_hist_clip)    # fit on *all* history

    if day in X.index:
        X_day_clip   = temp_clipper.transform(X.loc[[day]])
        X_day_scaled = temp_scaler.transform(X_day_clip)
        states[day]  = sjm.predict_online(
            pd.DataFrame(X_day_scaled,
                         index=[day],
                         columns=X.columns))[0]

    # assemble mini‑df for this factor & period
    out = pd.DataFrame({"state": pd.Series(states)},
                       index=list(states.keys()))
    out["ann_abs_ret"] = out["state"].map(abs_ret)
    return fac, out

def build_factor_views(factor_data_dict, saved_hyperparams, factors,
                       test_index,
                       refit_freq="MS", min_years=8, max_years=12,
                       init_start="2002-05-31"):

    views = {f:[] for f in factors}
    refit_dates = (test_index.to_series()
                   .resample(refit_freq)
                   .last()
                   .dropna())

    for j, refit_date in enumerate(refit_dates):
        if j < len(refit_dates)-1:
            next_refit = refit_dates.iloc[j+1]
        else:
            next_refit = test_index[-1]
        test_mask = (test_index>refit_date)&(test_index<=next_refit)
        test_chunk = test_index[test_mask]

        # ---- parallel over factors ----
        jobs = []
        for fac in factors:
            # latest hyperparams before refit_date
            hp_hist = [h for h in saved_hyperparams[fac]
                       if pd.to_datetime(h["date"])<=refit_date]
            if not hp_hist: continue
            hp = hp_hist[-1]
            jobs.append(delayed(_fit_one_factor)(
                fac, refit_date, test_chunk,
                factor_data_dict, hp,
                min_years, max_years, init_start))
        for fac, df in Parallel(n_jobs=-1)(jobs):
            views[fac].append(df)

    # concat & tidy
    for fac in factors:
        views[fac] = (pd.concat(views[fac])
                      .sort_index()
                      .loc[:,["state","ann_abs_ret"]])
    return views


# --------- build or load ----------         
if FORCE_REBUILD or not os.path.exists(VIEWS_FILE):
    factor_views = build_factor_views(factor_data_dict, saved_hyperparams, factors, 
                                      test_index,
                                      refit_freq="MS", 
                                      min_years=8, max_years=12, init_start="2002‑05‑31")
    with open(VIEWS_FILE, "wb") as f:
        pickle.dump(factor_views, f)
else:
    with open(VIEWS_FILE, "rb") as f:
        factor_views = pickle.load(f)



In [ ]:
# ------------------------------------------------------------
# ②  FAST BLACK‑LITTERMAN FUNCTION  (run as often as you like)
# ------------------------------------------------------------

def ewm_covariance(returns, halflife=126, min_periods=60):
    ewm_cov = returns.ewm(halflife=halflife,
                          adjust=False,
                          min_periods=min_periods).cov()
    if returns.empty: return pd.DataFrame()
    return ewm_cov.loc[returns.index[-1]]

def run_bl_once(views, returns_df,
                tau=0.05, delta=2.5,
                use_bl_cov=False,
                allow_market_short=False,
                allow_factor_short=False,     
                fallback_strategy="HOLD_RFR", # "HOLD_RFR", "SHORT_MARKET"
                tcost=0.00005):

    assets  = returns_df.columns.tolist()
    factors = list(views.keys())

    # ---------- per‑asset bounds ----------
    bounds = []
    for a in assets:
        if a == "Market":
            bounds.append((-1, 1) if allow_market_short else (0, 1))
        elif a == "rf":
            bounds.append((0, 1))            # no shorting cash
        else:
            bounds.append((-1, 1) if allow_factor_short else (0, 1))

    w = pd.DataFrame(index=returns_df.index, columns=assets, dtype=float)

    for t in returns_df.index[1:]:
        hist = returns_df.loc[:t].iloc[:-1]
        cov  = ewm_covariance(hist) * 252
        if cov.empty or cov.isna().any().any():
            continue

        q = {fac: views[fac].loc[t, "ann_abs_ret"] for fac in factors}

        bl = BlackLittermanModel(cov, pi="equal",
                                 tau=tau, delta=delta,
                                 absolute_views=q)

        cov_for_ef = bl.bl_cov() if use_bl_cov else cov

        ef = EfficientFrontier(bl.bl_returns(), cov_for_ef,
                               weight_bounds=bounds)

        try:
            ef.max_sharpe(risk_free_rate=returns_df.loc[t, "rf"])
            w.loc[t] = ef.clean_weights()

        except ValueError as e:
            if "expected return exceeding the risk-free rate" in str(e):
                if fallback_strategy == "HOLD_RFR":
                    w.loc[t] = 0.0; w.loc[t, "rf"] = 1.0
                elif fallback_strategy == "SHORT_MARKET":
                    w.loc[t] = 0.0; w.loc[t, "Market"] = -1.0
                else:
                    w.loc[t] = 0.0
            else:
                raise

    # ---------- P&L (+ optional costs) ----------
    pnl = (w.shift(1).fillna(0) * returns_df).sum(axis=1)
    if tcost > 0:
        pnl -= w.diff().abs().sum(axis=1).fillna(0) * tcost

    return w, pnl



In [ ]:
# ------------------------------------------------------------
# ③  QUICK EXPERIMENTS
# ------------------------------------------------------------
def annualized_sharpe(r):
    return (r.mean() / r.std()) * np.sqrt(252)

cfgs = [
    dict(tau=0.05, delta=2.5, use_bl_cov=False, allow_market_short=False, allow_factor_short=False),
    dict(tau=0.10, delta=2.5, use_bl_cov=False, allow_market_short=False, allow_factor_short=False),
]

for i,c in enumerate(cfgs,1):
    weights, rets = run_bl_once(factor_views, full_df, **c)
    print(f"Run{i}: Sharpe {annualized_sharpe(rets):.3f}")
